# Graph encoder

In [ ]:
import numpy as np
import pandas as pd
import random
import time
import os
import requests
from scipy import linalg

from sklearn.model_selection import RepeatedKFold
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
def EncoderLDA(A):
    begin1 = time.time()
    U, s, VT = np.linalg.svd(A)
    end1 = time.time()
    newx=([])
    dmax=50
    tmp = np.zeros([dmax,1])
    ASEKNN=[]

    #Split Data
    rkf = RepeatedKFold(n_splits=5, n_repeats=10, random_state=5)

    for train_index, test_index in rkf.split(A):
         #print("TRAIN:", train_index, "TEST:", test_index)
         X_train, X_test = A[train_index], A[test_index]
         y_train, y_test = y[train_index], y[test_index]
         X1 = X_train.transpose()
         X_train, X_test = X1[train_index], X1[test_index]
         for d in range (1,dmax):
            newx=U[:,0:d]*s[0:d]**0.5
            #newx=np.reshape(newx,(n,d))
            clf2 = LinearDiscriminantAnalysis()
            newx_train, newx_test =newx[train_index], newx[test_index]
            clf2.fit(newx_train, y_train.ravel())
            tmp[d,0]=tmp[d,0]+clf2.score(newx_test, y_test.ravel())/50
         
         neigh2 = KNeighborsClassifier(weights='distance',metric='euclidean')
         neigh2.fit(newx_train, y_train.ravel())
         ASEKNN.append(neigh2.score(newx_test, y_test))

    db=tmp.argmax()
    ASELDAACC=tmp.max()
    ASELDATIME=end1-begin1
    ASEKNNACC=sum(ASEKNN)/50
    #nk,w,Z
    begin2 = time.time()
    AEEKNNACC=[]
    AEELDAACC=[]
    AEEKNN2ACC=[]

    rkf = RepeatedKFold(n_splits=5, n_repeats=10, random_state=5)

    for train_index, test_index in rkf.split(A):
         #print("TRAIN:", train_index, "TEST:", test_index)
         X_train, X_test = A[train_index], A[test_index]
         y_train, y_test = y[train_index], y[test_index]
         X1 = X_train.transpose()
         X_train, X_test = X1[train_index], X1[test_index]
            
         begin4 = time.time()
         nk = np.zeros((1,K))

         for i in range(0,len(y_train)-1):
                nk[0,int(y_train[i,0]-1)]=nk[0,int(y_train[i,0]-1)]+1
                w = np.zeros((int(np.size(y_train)),K))

         for i in range(0,int(np.size(y_train))):
                k=int(y_train[i])
                w[i][k-1]=1/nk[0,k-1]

         z_train=np.matmul(X_train,w)
         z_test = np.matmul(X_test,w)
         end4 = time.time()
         zTIME = end4-begin4

         begin2 = time.time()
         neigh = KNeighborsClassifier(weights='uniform',metric='euclidean')
         neigh.fit(z_train, y_train.ravel())
         AEEKNNACC.append(neigh.score(z_test, y_test))
         end2 = time.time()
         AEEKNNTIME=end2-begin2
            
         neigh3 = KNeighborsClassifier(weights='distance',metric='euclidean')
         neigh.fit(z_train, y_train.ravel())
         AEEKNN2ACC.append(neigh.score(z_test, y_test))
            
         begin3 = time.time()
         clf4 = LinearDiscriminantAnalysis()
         clf4.fit(z_train,y_train.ravel())
         AEELDAACC.append(clf4.score(z_test, y_test.ravel()))
         end3 = time.time()
         AEELDATIME=end3-begin3

    #Get result
    AeK=sum(AEEKNNACC)/50
    AeL=sum(AEELDAACC)/50
    AeK2=sum(AEEKNN2ACC)/50
    Acc=np.array([ASELDAACC,ASEKNNACC,AeK,AeL,AeK2])
    Acc=np.round(Acc,3)
    Time=np.array([ASELDATIME,AEEKNNTIME+zTIME,AEELDATIME+zTIME])
    Time=np.round(Time,3)
    
    print(f"d = {db}")
    print(f"Accuracy for each method: ASELDA={Acc[0]},ASEKNN={Acc[1]},AEEKNN={Acc[2]},AEELDA={Acc[3]},AEEKNN2={Acc[4]}")
    print(f"Time for each method: {Time}")

### Datasets

In [ ]:
!git clone https://github.com/sparsh-ai/graph-embeddings.git

Cloning into 'graph-embeddings'...
remote: Enumerating objects: 83, done.
remote: Counting objects: 100% (83/83), done.
remote: Compressing objects: 100% (74/74), done.
remote: Total 83 (delta 7), reused 79 (delta 7), pack-reused 0
Unpacking objects: 100% (83/83), done.


In [ ]:
!apt-get -qq install tree
!tree --du -h ./graph-embeddings/data/bronze

Selecting previously unselected package tree.
(Reading database ... 155047 files and directories currently installed.)
Preparing to unpack .../tree_1.7.0-5_amd64.deb ...
Unpacking tree (1.7.0-5) ...
Setting up tree (1.7.0-5) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
./graph-embeddings/data/bronze
├── [ 61K]  Citeseer
│   ├── [ 45K]  citeseer edges.csv
│   ├── [9.6K]  citeseer node_labels.csv
│   └── [1.8K]  readme.html
├── [7.6M]  Cora
│   ├── [ 68K]  cora.cites
│   ├── [7.5M]  cora.content
│   ├── [ 53K]  cora edges.csv
│   └── [7.9K]  cora node_labels.csv
├── [ 11K]  DD244
│   ├── [6.6K]  DD244 Edges.csv
│   └── [ 758]  DD244 node_labels.csv
├── [384K]  EU-EMAIL-CORE
│   ├── [363K]  Core.xlsx
│   └── [ 17K]  Labels.xlsx
├── [ 24K]  Gene
│   ├── [ 15K]  gene edges.csv
│   ├── [3.2K]  gene node_labels.csv
│   └── [1.8K]  readme.html
├── [9.0K]  Internet Industry Partnerships
│   ├── [4.5K]  internet-industry-partnershipsedges.csv
│   └── [ 437]  internet-industry-partne

In [ ]:
data_basepath = '/content/graph-embeddings/data/bronze'

### CORA

In [ ]:
Edges = pd.read_csv(os.path.join(data_basepath,"Cora/cora edges.csv"),header=None)
Edges = np.array(Edges)

Labels = pd.read_csv(os.path.join(data_basepath,"Cora/cora node_labels.csv"),header=None)
y = np.array(Labels)

Edges.max(),np.shape(Edges),np.shape(y)

(2708, (5429, 2), (2708, 1))

In [ ]:
A = np.zeros((2708,2708))

for i in range (0,5429):
    A[Edges[i,0]-1,Edges[i,1]-1]=1

n = 2708
K = int(y.max())

In [ ]:
EncoderLDA(A)

d = 49
Accuracy for each method: ASELDA=0.4,ASEKNN=0.46,AEEKNN=0.697,AEELDA=0.725,AEEKNN2=0.697
Time for each method: [13.622  0.113  0.061]


### Citeseer

In [ ]:
Edges = pd.read_csv(os.path.join(data_basepath,"Citeseer/citeseer edges.csv"),header=None)
Edges = np.array(Edges)

Labels = pd.read_csv(os.path.join(data_basepath,"Citeseer/citeseer node_labels.csv"),header=None)
y = np.array(Labels)

Edges.max(),np.shape(Edges),np.shape(y)

(3264, (4536, 2), (3264, 1))

In [ ]:
A = np.zeros((3264,3264))

for i in range (0,4536):
    A[Edges[i,0]-1,Edges[i,1]-1]=1

n = 3264
K = int(y.max())

In [ ]:
EncoderLDA(A)

d = 47
Accuracy for each method: ASELDA=0.332,ASEKNN=0.349,AEEKNN=0.472,AEELDA=0.48,AEEKNN2=0.472
Time for each method: [23.402  0.117  0.057]


### EU EMail

In [ ]:
df = pd.read_excel(os.path.join(data_basepath,"EU-EMAIL-CORE/Labels.xlsx"), header = None)
y = df.to_numpy()

df2 = pd.read_excel(os.path.join(data_basepath,"EU-EMAIL-CORE/Core.xlsx"), header = None)
Edge = df2.to_numpy()

In [ ]:
A = np.zeros((1005,1005))

for i in range (0,25571):
    A[Edge[i,0],Edge[i,1]]=1

n=1005
K=int(y.max())

In [ ]:
EncoderLDA(A)

d = 48
Accuracy for each method: ASELDA=0.548,ASEKNN=0.585,AEEKNN=0.617,AEELDA=0.612,AEEKNN2=0.617
Time for each method: [0.784 0.135 0.087]


### DC-SBM

In [ ]:
def ADCSBM(n,k):
    A = np.zeros((n,n))
    theta = np.random.beta(1, 4, n)
    theta2 = theta/sum(theta)
    B=np.array([[0.9,0.1,0.1],[0.1,0.6,0.1],[0.1,0.1,0.3]])
    for i in range(0,n):
        for j in range(i+1,n):
            if y[i] == y[j] == 1:
                A[i,j]=np.random.binomial(1,theta[i]*theta[j]*B[0,0])
            elif y[i] == y[j] == 2:
                A[i,j]=np.random.binomial(1,theta[i]*theta[j]*B[1,1])
            elif y[i] == y[j] == 3:
                A[i,j]=np.random.binomial(1,theta[i]*theta[j]*B[2,2])
            else:
                A[i,j]=np.random.binomial(1,theta[i]*theta[j]*0.1)
    for i in range(0,n):
        for j in range(0,i):
            A[i,j]=A[j,i]
    return A

In [ ]:
def gety(n):
    y = np.zeros((n,1))
    #np.random.seed(2)
    for i in range(0,n):
        y[i,0] = np.random.choice(np.arange(1, 4), p=[0.2, 0.3,0.5])
    return(y)

In [ ]:
for n in range (100,1000,100):
    k=3
    y=gety(n)
    A=ADCSBM(n,k=k)
    EncoderLDA(A)

d = 25
Accuracy for each method: ASELDA=0.613,ASEKNN=0.544,AEEKNN=0.626,AEELDA=0.643,AEEKNN2=0.626
Time for each method: [0.003 0.004 0.003]
d = 18
Accuracy for each method: ASELDA=0.626,ASEKNN=0.483,AEEKNN=0.66,AEELDA=0.705,AEEKNN2=0.66
Time for each method: [0.017 0.007 0.006]
d = 12
Accuracy for each method: ASELDA=0.692,ASEKNN=0.591,AEEKNN=0.706,AEELDA=0.74,AEEKNN2=0.706
Time for each method: [0.029 0.011 0.008]
d = 14
Accuracy for each method: ASELDA=0.69,ASEKNN=0.581,AEEKNN=0.715,AEELDA=0.75,AEEKNN2=0.715
Time for each method: [0.062 0.025 0.021]
d = 32
Accuracy for each method: ASELDA=0.733,ASEKNN=0.653,AEEKNN=0.771,AEELDA=0.776,AEEKNN2=0.771
Time for each method: [0.113 0.032 0.023]
d = 28
Accuracy for each method: ASELDA=0.739,ASEKNN=0.591,AEEKNN=0.758,AEELDA=0.778,AEEKNN2=0.758
Time for each method: [0.191 0.045 0.041]
d = 25
Accuracy for each method: ASELDA=0.762,ASEKNN=0.641,AEEKNN=0.808,AEELDA=0.796,AEEKNN2=0.808
Time for each method: [0.303 0.043 0.036]
d = 30
Accuracy fo